# ¿Puede un puma alcanzar a un guanaco? - SIMULADOR INTERACTIVO

In [ ]:
# --- ⚙️ Instalar ipywidgets (solo en JupyterLite) ---
# ⚠️ EJECUTA ESTA CELDA PRIMERO. Solo se hace una vez por sesión.
import micropip
await micropip.install('ipywidgets')
print("✅ Paquete ipywidgets instalado. Ahora puedes ejecutar el simulador.")

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Button, Output
from IPython.display import display

# --- PARÁMETROS INICIALES ---
d0_init = 100.0
a_puma_init = 5.0
a_guanaco_init = 3.0
t_max_puma_init = 30.0

v_puma_max = 22.2   # 80 km/h
v_guanaco_max = 18.0 # 64.8 km/h

# --- FUNCIONES DE POSICIÓN ---
def calc_positions(d0, a_puma, a_guanaco, t_max):
    t = np.linspace(0, t_max, 300)
    t_puma_max_vel = min(v_puma_max / a_puma, t_max)
    t_guanaco_max_vel = min(v_guanaco_max / a_guanaco, t_max)

    x_puma = np.zeros_like(t)
    x_guanaco = np.zeros_like(t)

    for i, ti in enumerate(t):
        if ti <= t_puma_max_vel:
            x_puma[i] = 0.5 * a_puma * ti**2
        else:
            x_puma[i] = 0.5 * a_puma * t_puma_max_vel**2 + v_puma_max * (ti - t_puma_max_vel)

        if ti <= t_guanaco_max_vel:
            x_guanaco[i] = d0 + 0.5 * a_guanaco * ti**2
        else:
            x_guanaco[i] = d0 + 0.5 * a_guanaco * t_guanaco_max_vel**2 + v_guanaco_max * (ti - t_guanaco_max_vel)

    return t, x_puma, x_guanaco

# --- FUNCIÓN PARA DIBUJAR LA GRÁFICA ---
def plot_simulation(d0, a_puma, a_guanaco, t_max):
    t, x_puma, x_guanaco = calc_positions(d0, a_puma, a_guanaco, t_max)
    
    fig, ax = plt.subplots(figsize=(12, 7))
    
    line_puma, = ax.plot(t, x_puma, 'b-', linewidth=3, label='Puma')
    line_guanaco, = ax.plot(t, x_guanaco, 'r-', linewidth=3, label='Guanaco')
    ax.axvline(t_max, color='red', linestyle='--', label=f'Límite puma ({t_max:.1f}s)')
    
    # Buscar encuentro
    encuentro = None
    for i in range(len(t)):
        if x_puma[i] >= x_guanaco[i]:
            encuentro = (t[i], x_puma[i])
            break
    
    if encuentro:
        point_enc, = ax.plot(encuentro[0], encuentro[1], 'go', markersize=10, label=f'¡Encuentro! t={encuentro[0]:.2f}s')
    else:
        point_enc, = ax.plot([], [], 'go', markersize=10, label='Guanaco escapa')
    
    ax.set_title('¿Puede un puma alcanzar a un guanaco? - SIMULADOR INTERACTIVO', fontsize=14)
    ax.set_xlabel('Tiempo (s)')
    ax.set_ylabel('Posición (m)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    ymax = max(max(x_puma), max(x_guanaco)) * 1.1
    ax.set_ylim(0, ymax if ymax > 0 else 500)
    
    plt.tight_layout()
    plt.show()

# --- WIDGETS ---
d0_slider = FloatSlider(value=d0_init, min=50, max=200, step=1, description='Distancia inicial (m): ')
a_puma_slider = FloatSlider(value=a_puma_init, min=3, max=8, step=0.1, description='Acel. Puma (m/s²): ')
a_guanaco_slider = FloatSlider(value=a_guanaco_init, min=2, max=5, step=0.1, description='Acel. Guanaco (m/s²): ')
t_max_slider = FloatSlider(value=t_max_puma_init, min=20, max=60, step=1, description='Tiempo máx. puma (s): ')

button = Button(description='Actualizar gráfico', button_style='info')
reset_button = Button(description='Reiniciar valores', button_style='warning')

output = Output()

def on_button_click(b):
    with output:
        output.clear_output(wait=True)
        plot_simulation(
            d0_slider.value,
            a_puma_slider.value,
            a_guanaco_slider.value,
            t_max_slider.value
        )

def on_reset_click(b):
    d0_slider.value = d0_init
    a_puma_slider.value = a_puma_init
    a_guanaco_slider.value = a_guanaco_init
    t_max_slider.value = t_max_puma_init

button.on_click(on_button_click)
reset_button.on_click(on_reset_click)

display(d0_slider, a_puma_slider, a_guanaco_slider, t_max_slider, button, reset_button, output)

FloatSlider(value=100.0, description='Distancia inicial (m): ', max=200.0, min=50.0, step=1.0)

FloatSlider(value=5.0, description='Acel. Puma (m/s²): ', max=8.0, min=3.0)

FloatSlider(value=3.0, description='Acel. Guanaco (m/s²): ', max=5.0, min=2.0)

FloatSlider(value=30.0, description='Tiempo máx. puma (s): ', max=60.0, min=20.0, step=1.0)

Button(button_style='info', description='Actualizar gráfico', style=ButtonStyle())

Button(button_style='warning', description='Reiniciar valores', style=ButtonStyle())

Output()